In [1]:
import pandas as pd

vless_df = pd.read_excel('config_test_result.xlsx')

config = str(vless_df['json_config'].loc[3]).replace("'",'"').replace('True','true').replace('False','false')
print(config)

# Open file in write mode ('w' - creates new file or overwrites existing)
with open('/home/alireza/Desktop/proxy/config.json', 'w') as file:
    file.write(config)

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: "\s" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\s"? A raw string is also an option.
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


{"log": {"loglevel": "warning"}, "inbounds": [{"port": 10808, "listen": "127.0.0.1", "protocol": "socks", "settings": {"udp": true}}], "outbounds": [{"protocol": "vmess", "settings": {"vnext": [{"address": "s3.fastly4wards.com", "port": 2089, "users": [{"id": "014c4671-b0f9-4419-0ff5-5e01647ce34f", "alterId": 0, "security": "auto"}]}]}, "streamSettings": {"network": "tcp", "tcpSettings": {"header": {"type": "none"}}}}]}


In [2]:
import subprocess
import sys

# WARNING:
# Hardcoding sudo password is insecure.
# Acceptable only for personal/local automation.
PASSWORD = "6836\n"

def run_sudo_command(command):
    """
    Runs a sudo command and prints its output.
    """
    process = subprocess.Popen(
        ["sudo", "-S"] + command,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    out, err = process.communicate(PASSWORD)


run_sudo_command(["gsettings", "set", "org.gnome.system.proxy", "mode", "none"])
run_sudo_command(["sudo", "pkill", "xray"])

# 1. Bring up loopback interface
run_sudo_command(["ip", "link", "set", "lo", "up"])

# 2. Reload systemd units
run_sudo_command(["systemctl", "daemon-reload"])

# 3. Start and enable redsocks
run_sudo_command(["systemctl", "start", "redsocks"])
run_sudo_command(["systemctl", "enable", "redsocks"])


# 4. GNOME proxy settings (NO sudo here — must run as user)
def run_user_command(command):
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    out, err = process.communicate()


run_user_command(["gsettings", "set", "org.gnome.system.proxy", "mode", "manual"])

run_user_command(["gsettings", "set", "org.gnome.system.proxy.http", "host", "127.0.0.1"])
run_user_command(["gsettings", "set", "org.gnome.system.proxy.http", "port", "10808"])

run_user_command(["gsettings", "set", "org.gnome.system.proxy.https", "host", "127.0.0.1"])
run_user_command(["gsettings", "set", "org.gnome.system.proxy.https", "port", "10808"])

run_user_command(["gsettings", "set", "org.gnome.system.proxy.socks", "host", "127.0.0.1"])
run_user_command(["gsettings", "set", "org.gnome.system.proxy.socks", "port", "10808"])

run_user_command(["gsettings", "set", "org.gnome.system.proxy.ftp", "host", "127.0.0.1"])
run_user_command(["gsettings", "set", "org.gnome.system.proxy.ftp", "port", "10808"])



process = subprocess.Popen(
    ["sudo", "-S", "xray", "run", "-c", "/home/alireza/Desktop/proxy/config.json"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# Send sudo password
process.stdin.write(PASSWORD)
process.stdin.flush()

# Stream Xray output in real time
for line in process.stdout:
    sys.stdout.write(line)
    sys.stdout.flush()

process.wait()

Xray 26.1.23 (Xray, Penetrates Everything.) 0a42dba (go1.25.6 linux/amd64)
A unified platform for anti-censorship.
2026/02/06 15:47:28.231839 [Info] infra/conf/serial: Reading config: &{Name:/home/alireza/Desktop/proxy/config.json Format:json}
2026/02/06 15:47:28.234270 [Warning] common/errors: The feature VMess (with no Forward Secrecy, etc.) is deprecated, not recommended for using and might be removed. Please migrate to VLESS Encryption as soon as possible.
2026/02/06 15:47:28.240792 [Warning] core: Xray 26.1.23 started
2026/02/06 15:47:48.314085 from 127.0.0.1:48004 accepted //mobile.events.data.microsoft.com:443
2026/02/06 15:47:49.127771 from 127.0.0.1:48008 accepted //mobile.events.data.microsoft.com:443
2026/02/06 15:47:55.422009 from 127.0.0.1:42452 accepted //ab.chatgpt.com:443
2026/02/06 15:47:57.486752 from 127.0.0.1:42462 accepted //www.google.com:443
2026/02/06 15:47:58.769067 from 127.0.0.1:42478 accepted //www.google.com:443
2026/02/06 15:47:59.121138 from 127.0.0.1:424

KeyboardInterrupt: 

In [3]:
run_user_command(["gsettings", "set", "org.gnome.system.proxy", "mode", "none"])
run_user_command(["sudo", "pkill", "xray"])